<a href="https://colab.research.google.com/github/himanshugaur17/spend-classifier/blob/feature%2Finitial-setup/data_initial_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip --version


pip 22.0.4 from /usr/local/lib/python3.8/dist-packages/pip (python 3.8)


In [3]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
stmnt_df=pd.read_csv('/content/drive/MyDrive/Github/spend-classifier/feb-stmnt.csv')
stmnt_df

<class 'pandas.core.indexes.base.Index'>


In [7]:
stmnt_df.columns=list(map(lambda x:x.strip(),stmnt_df.columns.tolist()))
stmnt_df.columns

Index(['Txn Date', 'Value Date', 'Description', 'Ref No./Cheque No.', 'Debit',
       'Credit', 'Balance'],
      dtype='object')

In [29]:
def filterOutNoise(regex_filter,df,column):
  filtered_out_df=df[df[column].str.contains(regex_filter,regex=True)]
  return filtered_out_df.reset_index()

In [30]:
def myApplyFunc(x):
  return pd.to_numeric(x.str.replace(',',''),errors='coerce')

In [43]:
daily_debit_txns_df=filterOutNoise('TO TRANSFER|DEBIT(?! SWEEP)',stmnt_df,'Description')

daily_debit_txns_df[['Debit','Credit']]=daily_debit_txns_df[['Debit','Credit']].apply(myApplyFunc)
daily_debit_txns_df[daily_debit_txns_df['Debit']==daily_debit_txns_df['Debit'].max()]
total_debit=daily_debit_txns_df['Debit'].sum()
invstmnts_df=daily_debit_txns_df[daily_debit_txns_df['Description'].str.contains('DEBIT|Indian',regex=True)]
debit_df=pd.concat([invstmnts_df,daily_debit_txns_df]).drop_duplicates(ignore_index=True)
monthly_invstment_tot=invstmnts_df['Debit'].sum()

In [44]:
print(monthly_invstment_tot)

77500.0
